In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./archive/ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1')
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [3]:
i_cols = ['movie_id', 'title' ,'release date','video release date', 'IMDb URL',
'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
'War', 'Western']
movies = pd.read_csv('./archive/ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')
movies.head()

,movie_id,title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
movies = movies[['movie_id', 'title']]

In [5]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./archive/ml-100k/u.data', sep='\t', names=r_cols, encoding='latin-1')
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
ratings = ratings.drop('timestamp', axis=1)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X = ratings.copy()
y = ratings['user_id']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [9]:
from sklearn.metrics import mean_squared_error

In [10]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [11]:
def baseline(user_id, movie_id):
    return 3.0

In [12]:
def score(cf_model):
    id_pairs = zip(X_test['user_id'], X_test['movie_id'])
    y_pred = np.array([cf_model(user_id, movie_id) for user_id, movie_id in id_pairs])
    y_true = np.array(X_test['rating'])
    return rmse(y_true, y_pred)

In [13]:
score(baseline)

1.2470926188539486

# User based collaborative filtering

In [14]:
r_matrix = X_train.pivot_table(values='rating', index='user_id', columns='movie_id')

In [15]:
r_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1669,1670,1671,1673,1674,1675,1676,1679,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
def cf_user_mean(user_id, movie_id):
    if movie_id in r_matrix:
        mean_rating = r_matrix[movie_id].mean()
    else:
        mean_rating = 3.0
    return mean_rating

In [17]:
score(cf_user_mean)

1.0234701463131335

In [18]:
r_matrix_dummy = r_matrix.copy().fillna(0)

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

In [21]:
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

In [22]:
cosine_sim.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.118076,0.029097,0.011628,0.264677,0.312419,0.308729,0.224269,0.026017,0.286411,...,0.308475,0.055872,0.197862,0.131367,0.152449,0.084456,0.293293,0.056765,0.103536,0.326491
2,0.118076,1.000000,0.099097,0.107680,0.034279,0.152789,0.086705,0.078864,0.068940,0.092399,...,0.086927,0.259636,0.289092,0.318824,0.149105,0.186347,0.168034,0.106748,0.136796,0.080358
3,0.029097,0.099097,1.000000,0.252131,0.026893,0.062539,0.039767,0.089474,0.078162,0.037670,...,0.040918,0.019031,0.065417,0.055373,0.086503,0.018418,0.096993,0.109631,0.092574,0.018987
4,0.011628,0.107680,0.252131,1.000000,0.000000,0.045543,0.078812,0.095354,0.059498,0.053879,...,0.024226,0.050703,0.056561,0.107294,0.098892,0.000000,0.132900,0.142798,0.097066,0.015176
5,0.264677,0.034279,0.026893,0.000000,1.000000,0.202843,0.299619,0.163724,0.038474,0.153021,...,0.262547,0.048524,0.048312,0.022202,0.091910,0.066000,0.156172,0.115842,0.124297,0.267574


In [23]:
def  cf_user_wmean(user_id, movie_id):
    if movie_id in r_matrix:
        sim_score = cosine_sim[user_id]
        m_ratings =  r_matrix[movie_id]
        idx = m_ratings[m_ratings.isnull()].index
        m_ratings = m_ratings.dropna()
        sim_score = sim_score.drop(idx)
        wmean_rating = np.dot(sim_score, m_ratings)/sim_score.sum()
        
    else:
        wmean_rating = 3.0
    return wmean_rating

In [24]:
score(cf_user_wmean)

1.0174483808407588

In [25]:
merged_df = pd.merge(X_train, users)

In [26]:
merged_df.head()

,user_id,movie_id,rating,age,sex,occupation,zip_code
0,889,684,2,24,M,technician,78704
1,889,279,2,24,M,technician,78704
2,889,29,3,24,M,technician,78704
3,889,190,3,24,M,technician,78704
4,889,232,3,24,M,technician,78704


In [27]:
gender_mean = merged_df[['movie_id', 'sex', 'rating']].groupby(['movie_id','sex']).mean()

In [28]:
users = users.set_index('user_id')

In [29]:
def cf_gender(user_id, movie_id):
    if (movie_id in gender_mean) & (movie_id in r_matrix):
        gender = users.loc[user_id]['sex']
        if gender in gender_mean[movie_id]:
            gender_rating = gender_mean[movie_id][gender]
        else:
            gender_rating = 3.0
    else:
        gender_rating = 3.0
    return gender_rating

In [30]:
score(cf_gender)

1.2470926188539486

In [31]:
gen_occ_mean = merged_df[['sex', 'rating', 'movie_id', 'occupation']].pivot_table(values='rating', index='movie_id', columns=['occupation', 'sex'],aggfunc='mean')
gen_occ_mean.head()

occupation administrator           artist              doctor educator  \
sex                    F         M      F         M         M        F   
movie_id                                                                 
1                    4.0  4.222222   4.25  3.500000  3.666667     3.50   
2                    3.0  3.750000    NaN       NaN       NaN      NaN   
3                    3.5  2.500000    NaN       NaN       NaN     4.00   
4                    3.0  3.888889    NaN  4.666667  3.000000     2.75   
5                    4.0  2.333333    NaN       NaN       NaN     4.00   

occupation           engineer           entertainment  ... salesman            \
sex                M        F         M             F  ...        F         M   
movie_id                                               ...                      
1           3.923077      4.0  3.970588           5.0  ...      4.0  4.000000   
2           3.250000      NaN  3.363636           NaN  ...      NaN       NaN   
3           2.500000      NaN  3.625000           NaN  ...      NaN  1.000000   
4           3.636364      NaN  3.555556           NaN  ...      4.0  3.666667   
5           1.500000      NaN  2.666667           NaN  ...      NaN       NaN   

occupation scientist             student           technician            \
sex                F         M         F         M          F         M   
movie_id                                                                  
1                3.5  3.888889  3.833333  3.709091        4.0  4.200000   
2                NaN       NaN  2.333333  3.333333        NaN  2.714286   
3                NaN       NaN  2.000000  3.217391        NaN  4.000000   
4                NaN  3.600000  3.285714  3.724138        NaN  3.200000   
5                NaN  3.500000  4.333333  3.272727        NaN  3.333333   

occupation    writer            
sex                F         M  
movie_id                        
1           4.166667  3.142857  
2           5.000000  2.666667  
3                NaN  1.000000  
4           4.250000  3.500000  
5           4.000000  2.666667  

[5 rows x 41 columns]

In [32]:
def cf_gen_occ(user_id, movie_id):
    if (movie_id in gen_occ_mean) & (movie_id in r_matrix):
        user = users[user_id]
        
        occ = user['occupation']
        gen = user['sex']
        
        if occ in gen_occ_mean.loc[movie_id]:
            if gender in gen_occ_mean.loc[movie_id][occ]:
                rating = gen_occ_mean.loc[movie_id][occ][gender]
                if np.isnan(rating):
                    rating = 3.0
                return rating
    
    return 3.0

In [33]:
score(cf_gen_occ)

1.2470926188539486

In [34]:
from surprise import Reader, Dataset, KNNBasic
from surprise.model_selection import cross_validate

In [35]:
reader = Reader()

In [36]:
data = Dataset.load_from_df(ratings, reader)

In [37]:
knn = KNNBasic()

In [38]:
cross_validate(knn, data, measures=['RMSE'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.97437931, 0.97629424, 0.97719328, 0.98662819, 0.97873729]),
 'fit_time': (1.081200122833252,
  1.007720947265625,
  0.9776389598846436,
  1.0392181873321533,
  1.0041940212249756),
 'test_time': (6.421870946884155,
  6.321559190750122,
  6.698947429656982,
  6.692904710769653,
  6.54256796836853)}

In [39]:
from surprise import SVD

In [41]:
svd = SVD()

In [42]:
cross_validate(svd, data, measures=['RMSE'])

{'test_rmse': array([0.92474888, 0.93443894, 0.9377985 , 0.93891978, 0.94552226]),
 'fit_time': (9.817072629928589,
  9.596340656280518,
  9.92643928527832,
  9.798280954360962,
  9.811584711074829),
 'test_time': (0.2479548454284668,
  0.410388708114624,
  0.25191569328308105,
  0.28298091888427734,
  0.4049868583679199)}

In [43]:
ratings = pd.read_csv('./archive/ratings_small.csv')
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [44]:
svd = SVD()
trainset = data.build_full_trainset()

In [45]:
svd.fit(trainset)

In [46]:
id_map = pd.read_csv('./archive/movies_metadata.csv')
id_to_title = id_map.set_index('id')
title_to_id = id_map.set_index('title')

In [ ]:
cosine_sim_map = {}
for i in cosine_sim.index:
    title = id

In [93]:
def hybrid(userId, title):
    idx = title_to_id.loc[title]['id']
    tmdbId = title_to_id.loc[title]['id']
    movie_id = title_to_id.loc[title]['id']
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
#     print(movie_indices)
    movies = id_map.iloc[movie_indices][['title', 'vote_count', 'vote_average','id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, x).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [100]:
hybrid(1, 'Toy Story')

,title,vote_count,vote_average,id,est
428,Clean Slate,25.0,5.7,18658,2.688191
406,Martin Lawrence: You So Crazy,4.0,4.5,38129,2.688191
714,Heaven's Prisoners,17.0,5.0,36915,2.688191
96,Shopping,13.0,5.6,45549,2.688191
91,Vampire in Brooklyn,113.0,4.5,12158,2.688191
605,Jane Eyre,51.0,7.1,47333,2.688191
93,Broken Arrow,459.0,5.7,9208,2.688191
624,Denise Calls Up,5.0,6.6,47449,2.688191
338,The Baby-Sitters Club,23.0,5.3,48287,2.688191
312,Stargate,942.0,6.8,2164,2.688191


In [101]:
hybrid(2, 'Toy Story')

,title,vote_count,vote_average,id,est
428,Clean Slate,25.0,5.7,18658,3.460903
406,Martin Lawrence: You So Crazy,4.0,4.5,38129,3.460903
714,Heaven's Prisoners,17.0,5.0,36915,3.460903
96,Shopping,13.0,5.6,45549,3.460903
91,Vampire in Brooklyn,113.0,4.5,12158,3.460903
605,Jane Eyre,51.0,7.1,47333,3.460903
93,Broken Arrow,459.0,5.7,9208,3.460903
624,Denise Calls Up,5.0,6.6,47449,3.460903
338,The Baby-Sitters Club,23.0,5.3,48287,3.460903
312,Stargate,942.0,6.8,2164,3.460903
